# 🥋 Lekcja 10: Custom Autograd Function (Ręczne Pochodne)

PyTorch zna pochodne większości operacji matematycznych (`+`, `*`, `sin`, `exp`).
Ale czasem musisz stworzyć własną.

Aby to zrobić, tworzymy klasę dziedziczącą po `torch.autograd.Function` i implementujemy dwie metody statyczne:

1.  **`forward(ctx, input)`**:
    *   Robi obliczenia (np. $y = x^3$).
    *   Zapisuje dane potrzebne do pochodnej w **kontekście** (`ctx.save_for_backward`).
2.  **`backward(ctx, grad_output)`**:
    *   Dostaje gradient "z góry" (`grad_output`).
    *   Mnoży go przez naszą lokalną pochodną (Chain Rule).
    *   Zwraca gradient dla wejścia.

Zaimplementujemy własną funkcję sześcienną: $f(x) = x^3$.
Pochodna to $f'(x) = 3x^2$.

In [1]:
import torch

# Klasa musi dziedziczyć po torch.autograd.Function
class MyCube(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, x):
        # 1. Obliczenia Forward
        result = x ** 3
        
        # 2. Zapisywanie do pamięci (Context)
        # Musimy zapisać 'x', bo będzie potrzebne do policzenia pochodnej (3x^2)
        ctx.save_for_backward(x)
        
        return result

    @staticmethod
    def backward(ctx, grad_output):
        # grad_output: Gradient, który przyszedł z góry (od Loss function)
        
        # 1. Odzyskujemy zapisane tensory
        x, = ctx.saved_tensors
        
        # 2. Liczymy naszą lokalną pochodną: 3x^2
        local_grad = 3 * x ** 2
        
        # 3. Chain Rule: Mnożymy gradient z góry przez nasz
        grad_input = grad_output * local_grad
        
        return grad_input

# Tworzymy alias dla wygody (jak F.relu)
my_cube = MyCube.apply

print("Własna funkcja zdefiniowana.")

Własna funkcja zdefiniowana.


## Testowanie w Boju

Sprawdźmy, czy to działa.
1.  Przepuścimy dane przez naszą funkcję.
2.  Wywołamy `.backward()`.
3.  Sprawdzimy, czy `x.grad` zgadza się z matematyką.

Dla $x=2$:
*   Forward: $2^3 = 8$.
*   Backward: $3 \cdot 2^2 = 12$.

In [2]:
# Dane wejściowe (wymagają gradientu)
x = torch.tensor([2.0], requires_grad=True)

# Forward
y = my_cube(x)
print(f"Forward (2^3): {y.item()}")

# Backward
# Symulujemy, że to koniec sieci (Loss), więc gradient początkowy to 1.0
y.backward()

print(f"Gradient (3*2^2): {x.grad.item()}")

if x.grad.item() == 12.0:
    print("✅ Matematyka się zgadza!")
else:
    print("❌ Coś poszło nie tak.")

Forward (2^3): 8.0
Gradient (3*2^2): 12.0
✅ Matematyka się zgadza!


## `gradcheck`: Ostateczny Egzamin

Jako ludzie, mylimy się przy liczeniu pochodnych.
PyTorch ma wbudowane narzędzie **`gradcheck`**.

Robi ono dwie rzeczy:
1.  Liczy gradient Twoją metodą `backward` (Analitycznie).
2.  Liczy gradient numerycznie (metodą różnic skończonych: $\frac{f(x+h) - f(x)}{h}$).
3.  Porównuje wyniki.

Jeśli napiszesz zły wzór w `backward`, `gradcheck` to wykryje.

In [3]:
from torch.autograd import gradcheck

# Dane testowe (musi być double precision dla gradcheck)
test_input = torch.randn(20, 20, dtype=torch.double, requires_grad=True)

# Uruchamiamy test
# eps=1e-6 (małe przesunięcie h)
# atol=1e-4 (tolerancja błędu)
try:
    is_ok = gradcheck(my_cube, test_input, eps=1e-6, atol=1e-4)
    print(f"Czy gradcheck przeszedł? {is_ok}")
except Exception as e:
    print(f"🚫 Błąd: {e}")

Czy gradcheck przeszedł? True


## Hackowanie Gradientów (Gradient Reversal Layer)

Po co to robić, skoro `x**3` działa automatycznie?
Bo czasami chcemy **oszukać** matematykę.

Przykład: **Gradient Reversal Layer (GRL)**.
Używany w Domain Adaptation.
*   Forward: Zachowuje się jak identyczność ($y = x$).
*   Backward: Odwraca znak gradientu ($grad_{in} = -grad_{out}$).

Dzięki temu jedna część sieci uczy się dobrze klasyfikować, a druga część sieci "ogłupia" się celowo (np. żeby nie rozpoznawać, z jakiej domeny pochodzi zdjęcie).

Bez `autograd.Function` byś tego nie zrobił.

In [4]:
class GradientReversal(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        # Forward: Nic nie robimy (Identity)
        return x

    @staticmethod
    def backward(ctx, grad_output):
        # Backward: Odwracamy znak!
        # To sprawia, że wagi będą aktualizowane w PRZECIWNĄ stronę niż powinny (uciekają od optimum)
        return -grad_output

grad_reverse = GradientReversal.apply

# Test
x = torch.tensor([5.0], requires_grad=True)
y = grad_reverse(x)

# Loss = y (chcemy zminimalizować y)
# Normalnie gradient byłby +1 (zmniejsz x).
# Tutaj gradient powinien być -1 (zwiększ x).
y.backward()

print(f"Gradient po odwróceniu: {x.grad.item()}")

Gradient po odwróceniu: -1.0


## 🥋 Black Belt Summary

Pisanie własnych funkcji Autograd jest konieczne, gdy:
1.  **Niedóżniczkowalność:** Używasz biblioteki C++/Numpy w środku sieci, która nie jest PyTorchem (musisz ręcznie powiedzieć sieci, jak policzyć gradient przez tę "czarną skrzynkę").
2.  **Stabilność numeryczna:** Standardowy wzór wybucha (NaN), a Ty znasz wzór uproszczony (np. LogSoftmax).
3.  **Hackowanie:** Chcesz zmienić fizykę uczenia (GRL, Gradient Clipping wewnątrz warstwy).

Pamiętaj o `ctx.save_for_backward`! Bez tego `backward` nie będzie miał dostępu do danych z `forward`.